# Intermediate SAMURAI Tutorial

---

This interactive tutorial will demonstrate how parameter value choices affect the final SAMURAI analysis. SAMURAI is a variational analysis technique that is described in Bell et al. (2012), Foerster et al. (2014), Foerster and Bell (2017), Cha and Bell (2021), and other publications. The SAMURAI analysis yields a maximum likelihood estimate of the atmospheric state for a given set of observations and error estimates by minimizing a variational cost function. 

This tutorial will focus on a few scientific parameters in SAMURAI and how they can affect the application runtime, smoothing, mass continuity constraints, etc.

---


## Tutorial Overview

### 1. Setup 

#### 1.1 Directory organization 

SAMURAI uses one parameter file to run, in which are two parameters that set the input and output directories. The input directory contains the center file and any data that will be used in the analysis. All output files are written to the output directory.

<div>
<img src="../images/samurai_intermediate_struct.png" width="550"/>
</div>

#### 1.2 QC-ed input data, parameter files, and center files:

**a. Parameter Group #1: Skip and Stride**
* 20181010.cen
* samurai_params_skipstride
* ~850 QC-ed NOAA TDR Dorade swp files
  
**b. Parameter Group #2: Gaussian Filter Lengths and Spline Cutoffs && Parameter #3: ???**
* 20220606.cen
* samurai_params_filterspline
* cfrad.20220606_071238.512_to_20220606_071827.311_RCWF_SUR.nc
* cfrad.20220606_071250.353_to_20220606_071848.637_SPOL_PrecipSur1_SUR.nc
* cfrad.20220606_071345.000_to_20220606_071827.000_TEAM-R_SUR.nc
* cfrad.20220606_071505.096_to_20220606_072015.817_RCTP_SUR.nc
* terrain.hgt

*The QC process is not included in this tutorial. An example of one type of QC can be found in the QC tutorial and HawkEdit/solo are other tools to QC data.

For this tutorial, all center files are provided. The parameter files are also filled with most parameters. For these exercises, we will be modifying one or two parameters at a time.

#### 1.3 Note on task cells

This notebook uses two colored cells to indicate tasks.

<div class="alert alert-block alert-info"> <b>File Task: modify parameters in text files.</b> 

These text blocks help the user modify the parameter files or other functions in *external* text files.

</div>

<div class="alert alert-block alert-warning"> <b>Cell Task: run a command in Jupyter notebook cell.</b> 

These text blocks instruct the users to run a command *in* a cell within the Jupyter notebook. If you prefer, you are welcome to copy the commands (minus the ! symbol) into a terminal window.

</div>

#### 1.4 Environment and packages¶

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. 

In [ ]:
import os
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['RAW_DIR'] = '/home/jovyan/share/raw'
base_dir = os.environ['BASE_DIR']
!echo "Base directory: " $BASE_DIR

In [ ]:
# import plotting/data packages
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import xarray as xr
import matplotlib as mpl
import cartopy.crs as ccrs
from metpy.plots import ctables
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
mpl.rcParams['figure.dpi'] = 300

We will copy the required data to one directory as SAMURAI requires and organize the directories using the following commands:

In [ ]:
# make subdirectory within data for input and output directory
!rm -rf ${BASE_DIR}/data/samurai_int/input_precip
!mkdir -p ${BASE_DIR}/data/samurai_int/input_precip

!rm -rf ${BASE_DIR}/data/samurai_int/input_michael
!mkdir -p ${BASE_DIR}/data/samurai_int/input_michael

# create samurai output directories for each SAMURAI run
# !rm -rf ${BASE_DIR}/data/samurai_int/output_skipstride
!mkdir -p ${BASE_DIR}/data/samurai_int/output_skipstride

# !rm -rf ${BASE_DIR}/data/samurai_int/output_filterspline
!mkdir -p ${BASE_DIR}/data/samurai_int/output_filterspline

# !rm -rf ${BASE_DIR}/data/samurai_int/output_finale
!mkdir -p ${BASE_DIR}/data/samurai_int/output_finale

# copy raw files to individual input directory
!cp ${RAW_DIR}/samurai_int/input_precip/* ${BASE_DIR}/data/samurai_int/input_precip
!cp ${RAW_DIR}/samurai_int/input_michael/* ${BASE_DIR}/data/samurai_int/input_michael


### 2. SAMURAI Parameter Group #1: Skip and Stride

#### 2.1 Radar skip background
This option can be used to skip beams in the radar data. When set to ‘1’, all beams are used. This is primarily used to thin the data to decrease the computational burden when using a lot of radar data.

#### 2.2 Radar stride background
This option sets the number of gates over which averaging occurs along the beam. A stride of ‘1’ uses all data, and higher numbers average multiple gates of the given stride. This is used to thin the data for computational reasons, to reduce noise, or to reduce the spatial scale of the radar observations.

#### 2.3 Dynamic stride background
When set to ‘1’ the stride is dynamically determined based on the range. At short range, the minimum stride is set to the radar_stride value. As the range increases, the stride increases to try and approximate spherical pulse volumes.

| Parameter | Description |
| --- | --- |
|radar_skip|Skip rate for including data. A value of 1 means all beams are used.|
|radar_stride|Number of gates over which averaging occurs along a beam.|
|dynamic_stride|Determines whether or not the stride is dynamically determined based on range. 0 is False, 1 is True.|

<div class="alert alert-block alert-info"> <b>File Task: modify the parameter.</b> 

Using either the navigation menu on the left or the terminal (click the + button, choose the terminal under Other), open the parameter file (...) and find the line with the parameter. In the text file, use ⌘+f or Ctrl+F, depending on your computer. In the terminal using vi, you can search for strings in the command mode by typing "/" followed by the string.

Try running SAMURAI using a combination of these values:
* radar_skip: 3, 4, or 5 (typical values for more research are 1-2, but only run with 2 on the high power server - the low and medium power servers will crash &#128517;)
* radar_stride: 4 or 8
* dynamic_stride: 0 or 1

Look at the resulting analysis, but also compare how long it takes SAMURAI to run. In the SAMURAI application output you can look at the timing - look for "runTime: loadMetObs" and "runTime: Cost3D minimize."


</div>


#### 2.2 Run SAMURAI

<div class="alert alert-block alert-warning"> <b>Cell Task: run SAMURAI.</b> 

Run SAMURAI in the cell below with the modified parameter file by using the *-params* flag.

</div>




In [ ]:
# uncomment the line below to run (i.e., delete the #)
# !samurai -params ${BASE_DIR}/params/samurai_int/samurai_params_skipstride


If you'd like to run SAMURAI again using a different combination of parameters, we recommend creating a new parameter file and new output directory (which would also require editing the output directory in the parameter file).

#### 2.3 Plot SAMURAI results

#### Load the NetCDF file

In [ ]:
inDir_s = base_dir+"/data/samurai_int/output_skipstride/"
file_s = "samurai_XYZ_analysis.nc"
ds_radar_s = xr.open_dataset(inDir_s+file_s).squeeze()
ds_radar_s.load()

#### Let's use a function to shorten the code.

In [ ]:
def plot_samurai(plotting_lon_s, plotting_lat_s, plotting_var_s, plotting_var_u_s, plotting_var_v_s, ref_cmap, ref_norm, step=4):
    fig = plt.figure(figsize=(12,12))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines('50m', linewidth=1., zorder=2)
           
    cf1 = ax.pcolormesh( plotting_lon_s, plotting_lat_s, plotting_var_s
                        , cmap=ref_cmap, norm=ref_norm
                        , alpha=0.8, shading='auto'
                        , transform=ccrs.PlateCarree() 
                        )
    cf_q = ax.quiver( plotting_lon_s[::step], plotting_lat_s[::step]
                        , plotting_var_u_s[::step, ::step], plotting_var_v_s[::step, ::step]
                        , scale=1000, width=0.004
                        , color='k'
                        , transform=ccrs.PlateCarree() 
                        )
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    cbar_ax = fig.add_axes([0.95, 0.3, 0.02, 0.4])
    cbar = fig.colorbar(cf1, cax=cbar_ax, fraction=0.04)
    cbar.ax.tick_params(labelsize=14)
    cbar.ax.set_title('[dBZ]', fontsize=14, y=-0.1)

In [ ]:
## Set NWS reflectivity colorbar:
ref_norm, ref_cmap = ctables.registry.get_with_steps('NWSStormClearReflectivity', -20, 0.5)
plotting_alt=1.5 # altitude at 1.5 km
plotting_var_s = ds_radar_s.DBZ.sel(altitude=plotting_alt)
plotting_var_u_s = ds_radar_s.U.sel(altitude=plotting_alt).data
plotting_var_v_s = ds_radar_s.V.sel(altitude=plotting_alt).data
plotting_lon_s = ds_radar_s.longitude
plotting_lat_s = ds_radar_s.latitude

plot_samurai(plotting_lon_s, plotting_lat_s, plotting_var_s, plotting_var_u_s, plotting_var_v_s, ref_cmap, ref_norm, 4)


Load in FRACTL output

In [ ]:
# Read file into radar object
inDir_f = base_dir+"/data/wind_guided/output_frac/20090605/"
file_f = "ncf_20090605_220457.nc"
ds_radar_f = xr.open_dataset(inDir_f+file_f).squeeze()
ds_radar_f

In [ ]:
plotting_CN = ds_radar_f.conditionNumber.sel(z0=plotting_alt*1000)
plotting_CN = plotting_CN

CN_threshold = 10
dbz_goodcondition = np.where(plotting_CN < CN_threshold, plotting_var_s, np.nan)
u_goodcondition = np.where(plotting_CN < CN_threshold, plotting_var_u_s, np.nan)
v_goodcondition = np.where(plotting_CN < CN_threshold, plotting_var_v_s, np.nan)

plot_samurai(plotting_lon_s, plotting_lat_s, plotting_var_s, u_goodcondition, v_goodcondition, ref_cmap, ref_norm, 4)

### 3. SAMURAI Parameter Group #2: Gaussian Filters and Spline Cutoffs 

#### 3.1 Gaussian filter background
These numbers set the Gaussian recursive filter length scale in gridpoints. If set to ‘-1’, the recursive filter is turned off for the given dimension. The minimum recommended value is 2.0, with higher values corresponding to larger spatial influence of the observations. Higher values also result in a smoother analysis.

#### 3.2 Spline cutoff background
These numbers set the spline cutoff length scale in gridpoints. See Ooyama (2002) for a discussion of the spline cutoff length. The recommended value is 2.0.

Reference: 

Ooyama, K. V., 2002: The Cubic-Spline Transform Method: Basic Definitions and Tests in a 1D Single Domain. Mon. Wea. Rev., 130, 2392–2415, [link](https://doi.org/10.1175/1520-0493(2002)130<2392:TCSTMB>2.0.CO;2).

| Parameter | Description |
| --- | --- |
|i_filter_length, j_filter_length, k_filter_length|Filter lengths for each dimension, where the units are grid points.|
|i_spline_cutoff, j_spline_cutoff, k_spline_cutoff|Spline cutoff length scale in grid points.|

<div class="alert alert-block alert-info"> <b>File Task: modify the parameter.</b> 

Using either the navigation menu on the left or the terminal (click the + button, choose the terminal under Other), open the parameter file (...) and find the line with the parameter. In the text file, use ⌘+f or Ctrl+F, depending on your computer. In the terminal using vi, you can search for strings in the command mode by typing "/" followed by the string.

Try running SAMURAI using one of these sets of values:
* i_filter_length, j_filter_length, k_filter_length: <4, 4, 2> or <8, 8, 4> or <2, 2, 2>
* i_spline_cutoff, j_spline_cutoff, k_spline_cutoff: <8, 8, 4> or <12, 12, 8> or <2, 2, 2>

When you look at the SAMURAI output, look at the smoothness of the analysis with respect to the grid cell resolution.


</div>


#### 3.2 Run SAMURAI

<div class="alert alert-block alert-warning"> <b>Cell Task: run SAMURAI.</b> 

Run SAMURAI in the cell below with the modified parameter file by using the *-params* flag.

</div>




In [ ]:
# uncomment the line below to run
# !samurai -params ${BASE_DIR}/params/samurai_int/samurai_params_filterspline


#### 3.3 Plot SAMURAI results

#### Load the NetCDF file

In [ ]:
inDir_s_2 = base_dir+"/data/samurai_int/output_filterspline/"
file_s_2 = "samurai_XYZ_analysis.nc"
ds_radar_s_2 = xr.open_dataset(inDir_s_2+file_s_2).squeeze()
ds_radar_s_2.load()

In [ ]:
plotting_alt_2=1.5 # altitude at 1.5 km
plotting_var_s_2 = ds_radar_s_2.DBZ.sel(altitude=plotting_alt)
plotting_var_u_s_2 = ds_radar_s_2.U.sel(altitude=plotting_alt).data
plotting_var_v_s_2 = ds_radar_s_2.V.sel(altitude=plotting_alt).data
plotting_lon_s_2 = ds_radar_s_2.longitude
plotting_lat_s_2 = ds_radar_s_2.latitude

plot_samurai(plotting_lon_s_2, plotting_lat_s_2, plotting_var_s_2, plotting_var_u_s_2, plotting_var_v_s_2, ref_cmap, ref_norm, 4)


Load in FRACTL output

In [ ]:
# Read file into radar object
inDir_f_2 = "~/share/raw/samurai_int/fractl/precip/20220606/"
file_f_2 = "ncf_20220606_072015.nc"
ds_radar_f_2 = xr.open_dataset(inDir_f_2+file_f_2).squeeze()
ds_radar_f_2

In [ ]:
plotting_CN_2 = ds_radar_f_2.conditionNumber.sel(z0=plotting_alt_2*1000)
plotting_CN_2 = plotting_CN_2 

CN_threshold_2 = 10
dbz_goodcondition_2 = np.where(plotting_CN_2 < CN_threshold_2, plotting_var_s_2, np.nan)
u_goodcondition_2 = np.where(plotting_CN_2 < CN_threshold_2, plotting_var_u_s_2, np.nan)
v_goodcondition_2 = np.where(plotting_CN_2 < CN_threshold_2, plotting_var_v_s_2, np.nan)

plot_samurai(plotting_lon_s_2, plotting_lat_s_2, plotting_var_s_2, u_goodcondition_2, v_goodcondition_2, ref_cmap, ref_norm, 4)

#### 3.2 Run SAMURAI

<div class="alert alert-block alert-warning"> <b>Cell Task: run SAMURAI.</b> 

Run SAMURAI in the cell below with the modified parameter file by using the *-params* flag.

</div>




In [ ]:
# uncomment the line below to run
# !samurai -params ${BASE_DIR}/params/samurai_int/samurai_params_


#### 3.3 Plot SAMURAI results

#### Load the NetCDF file

In [ ]:
inDir_s_2 = base_dir+"/data/samurai_int/output_sam/"
file_s_2 = "samurai_XYZ_analysis.nc"
ds_radar_s_2 = xr.open_dataset(inDir_s_2+file_s_2).squeeze()
ds_radar_s_2.load()

#### Plot the output at 1.5 km altitude

In [ ]:
plotting_alt=1.5 # altitude at 1.5 km
plotting_var_s_2 = ds_radar_s_2.DBZ.sel(altitude=plotting_alt)
plotting_var_u_s_2 = ds_radar_s_2.U.sel(altitude=plotting_alt).data
plotting_var_v_s_2 = ds_radar_s_2.V.sel(altitude=plotting_alt).data
plotting_lon_s_2 = ds_radar_s_2.longitude
plotting_lat_s_2 = ds_radar_s_2.latitude

plot_samurai(plotting_lon_s_2, plotting_lat_s_2, plotting_var_s_2, plotting_var_u_s_2, plotting_var_v_s_2, ref_cmap_2, ref_norm_2, 4)


### 4. SAMURAI Parameter #3: Mass Continuity 

#### 4.1 dbz_pseudow_weight background

Setting this greater than ‘0’ will create pseudo-observations of the vertical velocity boundary condition based on the reflectivity. A non-zero value sets a weak constraint for zero vertical velocity at echo-top and at the surface. The number specifies the pseudo-error in m/s. Note that larger values set a weaker constraint, and smaller numbers greater than zero set a stronger constraint. Strong vertical velocity constraints at the surface and domain top can be set using boundary conditions, but this is provides a weak constraint at the surface and a variable height in the middle of the domain.


| Parameter | Description |
| --- | --- |
|dbz_pseudow_weight|Strength of the pseudo-error in m/s.|

<div class="alert alert-block alert-info"> <b>File Task: modify the parameter.</b> 

Using either the navigation menu on the left or the terminal (click the + button, choose the terminal under Other), open the parameter file (...) and find the line with the parameter. In the text file, use ⌘+f or Ctrl+F, depending on your computer. In the terminal using vi, you can search for strings in the command mode by typing "/" followed by the string.

Try running SAMURAI using one of these sets of values:
* dbz_pseudow_weight: 0, 1, or 10

</div>


#### 4.2 Run SAMURAI

<div class="alert alert-block alert-warning"> <b>Cell Task: run SAMURAI.</b> 

Run SAMURAI in the cell below with the modified parameter file by using the *-params* flag.

</div>




In [ ]:
# uncomment the line below to run
# !samurai -params ${BASE_DIR}/params/samurai_int/samurai_params_finale


#### 4.3 Plot SAMURAI results

#### Load the NetCDF file

In [ ]:
inDir_s_3 = base_dir+"/data/samurai_int/output_finale/"
file_s_3 = "samurai_XYZ_analysis.nc"
ds_radar_s_3 = xr.open_dataset(inDir_s_3+file_s_3).squeeze()
ds_radar_s_3.load()

#### Plot the output at 1.5 km altitude

In [ ]:
plotting_alt=1.5 # altitude at 1.5 km
plotting_var_s_3 = ds_radar_s_3.DBZ.sel(altitude=plotting_alt)
plotting_var_u_s_3 = ds_radar_s_3.U.sel(altitude=plotting_alt).data
plotting_var_v_s_3 = ds_radar_s_3.V.sel(altitude=plotting_alt).data
plotting_lon_s_3 = ds_radar_s_3.longitude
plotting_lat_s_3 = ds_radar_s_3.latitude

plot_samurai(plotting_lon_s_3, plotting_lat_s_3, plotting_var_s_3, plotting_var_u_s_3, plotting_var_v_s_3, ref_cmap_3, ref_norm_3, 4)


This case will use the same FRACTL output as the previous example, so we can just replace the wind vectors with the conditioned velocity.

In [ ]:
plot_samurai(plotting_lon_s_3, plotting_lat_s_3, plotting_var_s_3, u_goodcondition_2, v_goodcondition_2, ref_cmap, ref_norm, 4)
